# Main imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import sys
import re
import pandas as pd
import numpy as np
import _pickle as cPickle
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from matplotlib.patches import Circle, RegularPolygon
from matplotlib.path import Path
from matplotlib.projections.polar import PolarAxes
from matplotlib.projections import register_projection
from matplotlib.spines import Spine
from matplotlib.transforms import Affine2D
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

if 'SUMO_HOME' in os.environ:
    tools = os.path.join(os.environ['SUMO_HOME'], 'tools')
    sys.path.append(tools)
    import traci
else:
    sys.exit("please declare environment variable 'SUMO_HOME'")

from model_classes import ActorCriticNetwork, Agent, DQN

### Important constants and state dict

In [2]:
n_actions = 5981
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
experiment_dict = {
    'dqn_base': {
        'weight_path_1_layer': './resultados_centro_1mlp/escenario0/1_layer/policy_net_weights_experiment_ep_2999.pt',
        'weight_path_2_layer': './resultados_centro_1mlp/escenario0/2_layer/policy_net_weights_experiment_ep_2999.pt',
        'weight_path_3_layer': './resultados_centro_1mlp/escenario0/3_layer/policy_net_weights_experiment_ep_2999.pt',
        'config': "../sumo_simulation/sim_config/km2_centro/scenario/osm.sumocfg",
        'plot_name': 'dqn_scenario_0'
    },
    'dqn_case_2x': {
        'weight_path_1_layer': './resultados_centro_1mlp/escenario1/1_layer/policy_net_weights_experiment_ep_2999.pt',
        'weight_path_2_layer': './resultados_centro_1mlp/escenario1/2_layer/policy_net_weights_experiment_ep_2999.pt',
        'weight_path_3_layer': './resultados_centro_1mlp/escenario1/3_layer/policy_net_weights_experiment_ep_2999.pt',
        'config': "../sumo_simulation/sim_config/km2_centro/scenario_2/osm.sumocfg",
        'plot_name': 'dqn_scenario_2',
    },
    'dqn_case_4x': {
        'weight_path_1_layer': './resultados_centro_1mlp/escenario2/1_layer/policy_net_weights_experiment_ep_2999.pt',
        'weight_path_2_layer': './resultados_centro_1mlp/escenario2/2_layer/policy_net_weights_experiment_ep_2999.pt',
        'weight_path_3_layer': './resultados_centro_1mlp/escenario2/3_layer/policy_net_weights_experiment_ep_2999.pt',
        'config': "../sumo_simulation/sim_config/km2_centro/scenario_3/osm.sumocfg",
        'plot_name': 'dqn_scenario_3'
    },
    'pg_base': {
        'weight_path_1_layer': './resultados_pg/escenario0/1_layer/ac_weights_experiment_ep_2999.pt',
        'weight_path_2_layer': './resultados_pg/escenario0/2_layer/ac_weights_experiment_ep_2999.pt',
        'weight_path_3_layer': './resultados_pg/escenario0/3_layer/ac_weights_experiment_ep_2999.pt',
        'config': "../sumo_simulation/sim_config/km2_centro/scenario/osm.sumocfg",
        'plot_name': 'scenario_0_pg'
    },
    'pg_2x': {
        'weight_path_1_layer': './resultados_pg/escenario1/1_layer/ac_weights_experiment_ep_2999.pt',
        'weight_path_2_layer': './resultados_pg/escenario1/2_layer/ac_weights_experiment_ep_2999.pt',
        'weight_path_3_layer': './resultados_pg/escenario1/3_layer/ac_weights_experiment_ep_1293.pt',
        'config': "../sumo_simulation/sim_config/km2_centro/scenario_2/osm.sumocfg",
        'plot_name': 'scenario_1_pg'
    },
    'pg_4x': {
        'weight_path_1_layer': './resultados_pg/escenario2/1_layer/ac_weights_experiment_ep_2999.pt',
        'weight_path_2_layer': './resultados_pg/escenario2/2_layer/ac_weights_experiment_ep_2999.pt',
        'weight_path_3_layer': './resultados_pg/escenario2/3_layer/ac_weights_experiment_ep_2192.pt',
        'config': "../sumo_simulation/sim_config/km2_centro/scenario_3/osm.sumocfg",
        'plot_name': 'scenario_2_pg'
    }
}

### Helper functions

In [4]:
def load_dqn_sd(state_dict, num_layers):
    state_dict["mlp1.weight"] = state_dict['module.mlp1.weight']
    state_dict["mlp1.bias"] = state_dict['module.mlp1.bias']
    state_dict["head.weight"] = state_dict['module.head.weight']
    state_dict["head.bias"] = state_dict['module.head.bias']
    
    del state_dict['module.mlp1.weight'], state_dict['module.mlp1.bias'], state_dict['module.head.weight'], state_dict['module.head.bias']
    
    if num_layers == '1':
        policy_net = DQN(n_actions, False, False).to(device)
        
    elif num_layers == '2':
        policy_net = DQN(n_actions, True, False).to(device)
        state_dict["mlp2.weight"] = state_dict['module.mlp2.weight']
        state_dict["mlp2.bias"] = state_dict['module.mlp2.bias']
        
        del state_dict['module.mlp2.weight'], state_dict['module.mlp2.bias']
        
    else:
        policy_net = DQN(n_actions, True, True).to(device)
        state_dict["mlp2.weight"] = state_dict['module.mlp2.weight']
        state_dict["mlp2.bias"] = state_dict['module.mlp2.bias']
        state_dict["mlp3.weight"] = state_dict['module.mlp3.weight']
        state_dict["mlp3.bias"] = state_dict['module.mlp3.bias']
        
        del state_dict['module.mlp2.weight'], state_dict['module.mlp2.bias'], state_dict['module.mlp3.weight'], state_dict['module.mlp3.bias']
    

    policy_net.load_state_dict(state_dict)

    policy_net.eval()
    
    return policy_net

In [5]:
def load_pg_sd(state_dict, num_layers):
    if num_layers == '1':
        agent = Agent(alpha=0.001, input_dims=[6], gamma=0.001,
                  n_actions=n_actions, layer2_size=False, layer3_size=False)
   
    elif num_layers == '2':
        agent = Agent(alpha=0.001, input_dims=[6], gamma=0.001,
                  n_actions=n_actions, layer3_size=False)
    else:
        agent = Agent(alpha=0.001, input_dims=[6], gamma=0.001,
                  n_actions=n_actions)

    agent.actor_critic.load_state_dict(state_dict)

    agent.actor_critic.eval()
    
    return agent

### Main agent loops

In [6]:
def run_agent_simulation(key, weight_path, sumoCmd):
    
    num_layers = re.findall('\d', weight_path)
    
    if 'dqn' in key:
        num_layers = num_layers[2]
    else:
        num_layers = num_layers[1]
    
    state_dict = torch.load(weight_path)
    
    if 'dqn' in key:
        agent_kind = 'dqn'
        agent = load_dqn_sd(state_dict, num_layers)
    else:
        agent_kind = 'pg'
        agent = load_pg_sd(state_dict, num_layers)

    traci.start(sumoCmd)

    action_dict = cPickle.load(open('../sumo_simulation/input/action_to_zone_km2_lince.pkl', 'rb'))
    state = torch.zeros([1,6], device=device)
    traci_ep = 0
    lane_id_list = traci.lane.getIDList()

    states_agent = []
    states_agent_mean = []
    truck_emissions_agent = []

    for e in range(86400):

        if traci_ep % 3600 == 0 and traci_ep != 0:

            #Start agent interaction
            
            if agent_kind == 'dqn':
                action = agent(state)
            else:
                action = agent.choose_action(state)

            #Apply regulation and run steps
            reg_action = action > 0

            #print(reg_action.view(-1))

            for index, lane_id in enumerate(reg_action.view(-1)):
                        #for lane_id in lane_indices:
                if lane_id.item() == 1:
                    if action_dict[index] is not None:
                        traci.lane.setDisallowed(action_dict[index], ['truck'])
                    else:
                        pass
                else:
                    if action_dict[index] is not None:
                        traci.lane.setAllowed(action_dict[index], ['truck'])
                    else:
                        pass    

            vehicle_id_list = traci.vehicle.getIDList()
            
            vehicle_types = [traci.vehicle.getTypeID(v_id) for v_id in vehicle_id_list]
            vehicle_co2 = [traci.vehicle.getCO2Emission(v_id) for i, v_id in enumerate(vehicle_id_list) 
                                      if 'truck' in vehicle_types[i]]

            try:
                truck_emissions_agent.append(sum(vehicle_co2)/len(vehicle_co2))
            except:
                truck_emissions_agent.append(0)

            #Get simulation values
            co2 = [traci.lane.getCO2Emission(edge_id) for edge_id in lane_id_list]
            co = [traci.lane.getCOEmission(edge_id) for edge_id in lane_id_list]
            nox = [traci.lane.getNOxEmission(edge_id) for edge_id in lane_id_list]
            pmx = [traci.lane.getPMxEmission(edge_id) for edge_id in lane_id_list]
            noise = [traci.lane.getNoiseEmission(edge_id) for edge_id in lane_id_list]
            fuel = [traci.lane.getFuelConsumption(edge_id) for edge_id in lane_id_list]

            sim_results = np.array([co2, co, pmx, nox, noise, fuel])

            next_state = np.transpose(sim_results).mean(axis=0)
            
            states_agent.append(np.transpose(sim_results).sum(axis=0))
            states_agent_mean.append(next_state)

            next_state = torch.from_numpy(next_state).to(device).float()

            state += next_state

        traci.simulationStep()
        traci_ep += 1
    traci.close(False)
    
    if agent_kind == 'dqn':
        values = [agent(torch.from_numpy(state).float().to(device).view(-1,6)) 
                  for state in states_agent_mean]
    else:
        values = [agent.choose_action(torch.from_numpy(state).float().to(device).view(-1,6)) 
                  for state in states_agent_mean]
    
    
    #values = torch.cat(values).view(-1).detach().cpu().numpy()
    
    return states_agent, truck_emissions_agent, values

### Result Dictionary to store results

In [7]:
result_dict = {'{}_{}_layers'.format(algorithm, layer):'' 
               for algorithm in experiment_dict.keys() for layer in range(1,4)}

for key in result_dict.keys():
    result_dict[key] = {k: '' for k in ['cauchy', 'chi_squared', 'gaussian']}

### Main loop for tests

In [8]:
for key in experiment_dict.keys():
    for path in experiment_dict[key]:
        if 'weight' in path:
            for test in ['sc0', 'sc1', 'sc2']:
                sumoCmd = ['/usr/bin/sumo/bin/sumo','-c',
             '/home/andres/Documents/tesis_pregrado/sumo_simulation/sim_config/km2_lince/geo_test/{}/osm.sumocfg'.format(test),
             '-e', '86400']
                print(key, path, test)
                result_dict['_'.join([key,path.split('_')[2], 'layers'])][test] = run_agent_simulation(key, experiment_dict[key][path], sumoCmd)

dqn_base weight_path_1_layer sc0
 Retrying in 1 seconds
dqn_base weight_path_1_layer sc1
 Retrying in 1 seconds
dqn_base weight_path_1_layer sc2
 Retrying in 1 seconds
dqn_base weight_path_2_layer sc0
 Retrying in 1 seconds
dqn_base weight_path_2_layer sc1
 Retrying in 1 seconds
dqn_base weight_path_2_layer sc2
 Retrying in 1 seconds
dqn_base weight_path_3_layer sc0
 Retrying in 1 seconds
dqn_base weight_path_3_layer sc1
 Retrying in 1 seconds
dqn_base weight_path_3_layer sc2
 Retrying in 1 seconds
dqn_case_2x weight_path_1_layer sc0
 Retrying in 1 seconds
dqn_case_2x weight_path_1_layer sc1
 Retrying in 1 seconds
dqn_case_2x weight_path_1_layer sc2
 Retrying in 1 seconds
dqn_case_2x weight_path_2_layer sc0
 Retrying in 1 seconds
dqn_case_2x weight_path_2_layer sc1
 Retrying in 1 seconds
dqn_case_2x weight_path_2_layer sc2
 Retrying in 1 seconds
dqn_case_2x weight_path_3_layer sc0
 Retrying in 1 seconds
dqn_case_2x weight_path_3_layer sc1
 Retrying in 1 seconds
dqn_case_2x weight_path_

/home/andres/Documents/tesis_pregrado/notebooks/model_classes.py:68: UserWarning:

Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.



pg_base weight_path_1_layer sc1
 Retrying in 1 seconds
pg_base weight_path_1_layer sc2
 Retrying in 1 seconds
pg_base weight_path_2_layer sc0
 Retrying in 1 seconds
pg_base weight_path_2_layer sc1
 Retrying in 1 seconds
pg_base weight_path_2_layer sc2
 Retrying in 1 seconds
pg_base weight_path_3_layer sc0
 Retrying in 1 seconds
pg_base weight_path_3_layer sc1
 Retrying in 1 seconds
pg_base weight_path_3_layer sc2
 Retrying in 1 seconds
pg_2x weight_path_1_layer sc0
 Retrying in 1 seconds
pg_2x weight_path_1_layer sc1
 Retrying in 1 seconds
pg_2x weight_path_1_layer sc2
 Retrying in 1 seconds
pg_2x weight_path_2_layer sc0
 Retrying in 1 seconds
pg_2x weight_path_2_layer sc1
 Retrying in 1 seconds
pg_2x weight_path_2_layer sc2
 Retrying in 1 seconds
pg_2x weight_path_3_layer sc0
 Retrying in 1 seconds
pg_2x weight_path_3_layer sc1
 Retrying in 1 seconds
pg_2x weight_path_3_layer sc2
 Retrying in 1 seconds
pg_4x weight_path_1_layer sc0
 Retrying in 1 seconds
pg_4x weight_path_1_layer sc1


### Result Values

In [9]:
for test in ['sc0', 'sc1', 'sc2']:
    for key in result_dict.keys():
        print(key, test)
        np.save('arrays_lince/{}_{}.npy'.format(key,test), np.array(result_dict[key][test][0]).sum(axis=0))
        print(list(np.array(result_dict[key][test][0]).sum(axis=0)))

dqn_base_1_layers sc0
[91452.76756121367, 3565.7767360946473, 7.833935455508666, 147.42298569655648, 2425.3578619177997, 39.27519862516043]
dqn_base_2_layers sc0
[63043.72343700901, 2911.16368628881, 3.7219239059087044, 35.32230831275942, 2030.9862281329147, 27.103266425250762]
dqn_base_3_layers sc0
[68508.71518001941, 3060.534034106314, 3.616620062660844, 31.17153827556672, 2178.6091576837175, 29.45288227268902]
dqn_case_2x_1_layers sc0
[91452.76756121367, 3565.7767360946473, 7.833935455508666, 147.42298569655648, 2425.3578619177997, 39.27519862516043]
dqn_case_2x_2_layers sc0
[53624.47078622594, 1952.6567758655626, 2.5713057174349903, 13.243904084708557, 1781.4750658663932, 23.05368459036666]
dqn_case_2x_3_layers sc0
[74149.74092969018, 2501.349823140923, 3.337538260855079, 28.994163950541445, 1876.9394352586723, 31.877260521631772]
dqn_case_4x_1_layers sc0
[108016.41852289342, 5293.35713282501, 10.873444706012195, 188.8254284915515, 2493.488065767306, 46.39656948177082]
dqn_case_4x_

In [16]:
for key in result_dict.keys():
    print(key)
    print(np.array(result_dict[key]['sc0'][0]).sum(axis=0))

dqn_base_1_layers
[9.14527676e+04 3.56577674e+03 7.83393546e+00 1.47422986e+02
 2.42535786e+03 3.92751986e+01]
dqn_base_2_layers
[6.30437234e+04 2.91116369e+03 3.72192391e+00 3.53223083e+01
 2.03098623e+03 2.71032664e+01]
dqn_base_3_layers
[6.85087152e+04 3.06053403e+03 3.61662006e+00 3.11715383e+01
 2.17860916e+03 2.94528823e+01]
dqn_case_2x_1_layers
[9.14527676e+04 3.56577674e+03 7.83393546e+00 1.47422986e+02
 2.42535786e+03 3.92751986e+01]
dqn_case_2x_2_layers
[5.36244708e+04 1.95265678e+03 2.57130572e+00 1.32439041e+01
 1.78147507e+03 2.30536846e+01]
dqn_case_2x_3_layers
[7.41497409e+04 2.50134982e+03 3.33753826e+00 2.89941640e+01
 1.87693944e+03 3.18772605e+01]
dqn_case_4x_1_layers
[1.08016419e+05 5.29335713e+03 1.08734447e+01 1.88825428e+02
 2.49348807e+03 4.63965695e+01]
dqn_case_4x_2_layers
[5.36244708e+04 1.95265678e+03 2.57130572e+00 1.32439041e+01
 1.78147507e+03 2.30536846e+01]
dqn_case_4x_3_layers
[7.41497409e+04 2.50134982e+03 3.33753826e+00 2.89941640e+01
 1.87693944e+03

In [18]:
for test in ['sc0', 'sc1', 'sc2']:
    d = pd.DataFrame([np.array(result_dict[key][test][0]).sum(axis=0) for key in result_dict.keys()])
    print(d.to_latex())

\begin{tabular}{lrrrrrr}
\toprule
{} &              0 &            1 &          2 &           3 &            4 &          5 \\
\midrule
0  &   91452.767561 &  3565.776736 &   7.833935 &  147.422986 &  2425.357862 &  39.275199 \\
1  &   63043.723437 &  2911.163686 &   3.721924 &   35.322308 &  2030.986228 &  27.103266 \\
2  &   68508.715180 &  3060.534034 &   3.616620 &   31.171538 &  2178.609158 &  29.452882 \\
3  &   91452.767561 &  3565.776736 &   7.833935 &  147.422986 &  2425.357862 &  39.275199 \\
4  &   53624.470786 &  1952.656776 &   2.571306 &   13.243904 &  1781.475066 &  23.053685 \\
5  &   74149.740930 &  2501.349823 &   3.337538 &   28.994164 &  1876.939435 &  31.877261 \\
6  &  108016.418523 &  5293.357133 &  10.873445 &  188.825428 &  2493.488066 &  46.396569 \\
7  &   53624.470786 &  1952.656776 &   2.571306 &   13.243904 &  1781.475066 &  23.053685 \\
8  &   74149.740930 &  2501.349823 &   3.337538 &   28.994164 &  1876.939435 &  31.877261 \\
9  &   68473.120376 &  2438